# Semantic Parsing Final Project
Link to the paper: https://aclanthology.org/P16-1004.pdf

Read through the paper fully before starting the assignment!

In [17]:
import torch
import torch.nn as nn

from google.colab import drive
drive.mount('/content/drive')

FILEPATH = "drive/MyDrive/data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Downloading
This cell obtains the pre-processed Jobs dataset (see the paper) that you will be using to train and evaluate your model. (Pre-processed meaning that argument identification, section 3.6, has already been done for you). You should only need to run this cell ***once***. Feel free to delete it after running. Create a folder in your Google Drive in which the code below will store the pre-processed data needed for this project. Modify `FILEPATH` above to direct to said folder. It should start with `drive/MyDrive/...`, feel free to take a look at previous assignments that use mounting Google Drive if you can't remember what it should look like. *Make sure the data path ends with a slash character ('/').* The below code will access the zip file containing the pre-processed Jobs dataset from the paper and extract the files into your folder! Feel free to take a look at the `train.txt` and `test.txt` files to see what the data looks like. :)

# Data Pre-processing
The following code is defined for you! It extracts the queries (inputs to your Seq2Seq model) and logical forms (expected outputs) from the training and testing files. It also does important pre-processing such as padding the queries and logical forms and turns the words into vocab indices. **Look over and understand this code before you start the assignment!**

In [19]:
def extract_file(filename):
  """
  Extracts queries and corresponding logical forms from either
  train.txt or test.txt. (Feel free to take a look at the files themselves
  in your Drive!)

  Parameters
  ----------
  filename : str
      name of the file to extract from

  Returns
  ----------
  tuple[list[list[str]], list[list[str]]]
      a tuple of a list of queries and their corresponding logical forms
      each in the form of a list of string tokens
  """
  queries, logical_forms = [], []
  with open(FILEPATH + filename) as f:
    for line in f:
      line = line.strip() # remove new line character
      query, logical_form = line.split('\t')

      query = query.split(' ')[::-1] # reversed inputs are used the paper (section 4.2)
      logical_form = ["<s>"] + logical_form.split(' ') + ["</s>"]

      queries.append(query)
      logical_forms.append(logical_form)
  return queries, logical_forms

query_train, lf_train = extract_file('train.txt') # 500 instances
query_test, lf_test = extract_file('test.txt') # 140 instances

In [20]:
from collections import Counter

query_vocab = Counter()
for l in query_train:
  query_vocab.update(l)

query_word2idx = {}
for w, c in query_vocab.items():
  if c >= 2:
    query_word2idx[w] = len(query_word2idx)
query_word2idx['<UNK>'] = len(query_word2idx)
query_word2idx['<PAD>'] = len(query_word2idx)
query_idx2word = {i:word for word,i in query_word2idx.items()}

query_vocab = list(query_word2idx.keys())

lf_vocab = Counter()
for lf in lf_train:
  lf_vocab.update(lf)

lf_vocab['<UNK>'] = 0
lf_vocab['<PAD>'] = 0
lf_idx2word = {i:word for i, word in enumerate(lf_vocab.keys())}
lf_word2idx = {word:i for i, word in lf_idx2word.items()}

In [21]:
query_train_tokens = [[query_word2idx.get(w, query_word2idx['<UNK>']) for w in l] for l in query_train]
query_test_tokens = [[query_word2idx.get(w, query_word2idx['<UNK>']) for w in l] for l in query_test]

lf_train_tokens = [[lf_word2idx.get(w, lf_word2idx['<UNK>']) for w in l] for l in lf_train]
lf_test_tokens = [[lf_word2idx.get(w, lf_word2idx['<UNK>']) for w in l] for l in lf_test]

def pad(seq, max_len, pad_token_idx):
  """
  Pads a given sequence to the max length using the given padding token index

  Parameters
  ----------
  seq : list[int]
      sequence in the form of a list of vocab indices
  max_len : int
      length sequence should be padded to
  pad_token_idx
      vocabulary index of the padding token

  Returns
  ----------
  list[int]
      padded sequence
  """
  seq = seq[:max_len]
  padded_seq = seq + (max_len - len(seq)) * [pad_token_idx]
  return padded_seq

query_max_target_len = max([len(i) for i in query_train_tokens])
query_train_tokens = [pad(i, query_max_target_len, query_word2idx['<PAD>']) for i in query_train_tokens]
query_test_tokens = [pad(i, query_max_target_len, query_word2idx['<PAD>']) for i in query_test_tokens]

lf_max_target_len = int(max([len(i) for i in lf_train_tokens]) * 1.5)
lf_train_tokens = [pad(i, lf_max_target_len, lf_word2idx['<PAD>']) for i in lf_train_tokens]
lf_test_tokens = [pad(i, lf_max_target_len, lf_word2idx['<PAD>']) for i in lf_test_tokens]

# Data Loading
The following code creates a JobsDataset and DataLoaders to use with your implemented model. Take a look at the main function at the end of this stencil to see how they are used in context.

In [22]:
from torch.utils.data import Dataset, DataLoader, default_collate

class JobsDataset(Dataset):
  """Defines a Dataset object for the Jobs dataset to be used with Dataloader"""
  def __init__(self, queries, logical_forms):
    """
    Initializes a JobsDataset

    Parameters
    ----------
    queries : list[list[int]]
        a list of queries, which have been tokenized and padded, in the form
        of a list of vocab indices
    logical_forms : list[list[int]]
        a list of corresponding logical forms, which have been tokenized and
        padded, in the form of a list of vocab indices
    """
    self.queries = queries
    self.logical_forms = logical_forms

  def __len__(self) -> int:
    """
    Returns the amount of paired queries and logical forms in the dataset

    Returns
    ----------
    int
        length of the dataset
    """
    return len(self.queries)

  def __getitem__(self, idx: int) -> tuple[list[int], list[int]]:
    """
    Returns a paired query and logical form at the specified index

    Parameters
    ----------
    idx : int
        specified index of the dataset

    Returns
    ----------
    tuple[list[int], list[int]]
        paired query and logical form at the specified index, in the form of
        a list of vocab indices
    """
    return self.queries[idx], self.logical_forms[idx]

def build_datasets() -> tuple[JobsDataset, JobsDataset]:
  """
  Builds a train and a test dataset from the queries and logical forms
  train and test tokens

  Returns
  ----------
  tuple[JobsDataset, JobsDataset]
      a training and testing JobsDataset
  """
  jobs_train = JobsDataset(queries=query_train_tokens, logical_forms=lf_train_tokens)
  jobs_test = JobsDataset(queries=query_test_tokens, logical_forms=lf_test_tokens)
  return jobs_train, jobs_test

def collate(batch : list[tuple[list[int], list[int]]]) -> tuple[torch.Tensor, torch.Tensor]:
  """
  Used as collate_fn when creating the Dataloaders from the dataset

  Parameters
  ----------
  batch : list[tuple[list[int], list[int]]]
      a list of outputs of __getitem__

  Returns
  ----------
  tuple[torch.Tensor, torch.Tensor]
      a batched set of input sequences and a batched set of target sequences
  """
  src, tgt = default_collate(batch)
  return torch.stack(src), torch.stack(tgt)

def build_dataloaders(dataset_train: JobsDataset, dataset_test: JobsDataset,
                      train_batch_size: int) -> tuple[DataLoader, DataLoader]:
  """
  Used as collate_fn when creating the Dataloaders from the dataset, batching
  the training data according to the inputted batch size and batching the
  testing data with a batch size of 1

  Parameters
  ----------
  dataset_train : JobsDataset
      training dataset
  dataset_test : JobsDataset
      testing dataset
  train_batch_size : int
      batch size to be used during training

  Returns
  ----------
  tuple[DataLoader, DataLoader]
      a training and testing DataLoader
  """
  dataloader_train = DataLoader(dataset_train, batch_size=train_batch_size, shuffle=True, collate_fn=collate)
  dataloader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=collate)
  return dataloader_train, dataloader_test

# TODO: Define your model here!

In [23]:
QUERY_VOCAB_LEN = len(query_vocab)
LF_VOCAB_LEN = len(lf_vocab)

In [24]:
class Encoder(nn.Module):
    def __init__(self, vocab_size: int, hidden_size: int, embed_size: int, layers: int, drop_rate: float):
        """
        Initializes an Encoder module

        Parameters
        ----------
        vocab_size : int
            size of the encoder input vocabulary
        hidden_size : int
            size of the hidden states for the LSTM
        embed_size : int
            size of the embeddings for each word
        layers : int
            number of layers of the LSTM
        drop_rate : float
            rate at which weights are probabalistically excluded from the model
        """
        super(Encoder, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_size)
        self.lstm_layers = nn.LSTM(input_size=embed_size, hidden_size=hidden_size, num_layers=layers, dropout=drop_rate, batch_first=True)

    def forward(self, x: torch.Tensor):
        """
        Feeds a tensor through the Encoder

        Parameters
        ----------
        x : int
            input tensor for the Encoder

        Returns
        ----------
        tuple[torch.Tensor, torch.Tensor, torch.Tensor]
            the output, hidden_state, and cell state that is outputted from the LSTM layers
        """
        emb = self.embedding_layer(x) # (B x L x E)
        output, (hidden_state, cell_state) = self.lstm_layers(emb) # Output: (B x L x H), HS: (L x B x H)
        return output, hidden_state, cell_state

In [49]:
class Attention(nn.Module):
    def __init__(self, n: int):
        """
        Initializes an Attention module

        Parameters
        ----------
        n : int
            dimensions of the Linear Layers used in the attention calculations (should be the size of hidden_state for matrix mult. to work)
        """
        super(Attention, self).__init__()
        self.softmax = nn.Softmax(dim=1)
        self.W1 = nn.Linear(n, n) # dimensions: (hidden_size, hidden_size) bc matrix mult with h
        self.W2 = nn.Linear(n, n) # dimensions: (hidden_size, hidden_size) bc matrix mult with c
        self.tanh = nn.Tanh()

    def forward(self, enc_outputs: torch.Tensor, dec_hidden_state: torch.Tensor):
        """
        Feeds Encoder outputs and a Decoder hidden state through the Encoder

        Parameters
        ----------
        enc_outputs : torch.Tensor
            outputs of the encoder which show the end states at every time step in the LSTM
        dec_hidden_state : torch.Tensor
            current hidden state from the decoder
        Returns
        ----------
        torch.Tensor
            the new hidden state after attention is taken in from the encoder
        """
        # *Print statements for video*
        #print("--------------------------------")
        # enc_outputs: 21 x 20 x 256 B x S x H
        # print("Starting Encoder Out Dim", enc_outputs.size())
        # print("Starting Hidden State Dim", dec_hidden_state.size())

        dec_hidden_state = dec_hidden_state.permute(1, 2, 0) # B x H x L
        dec_hidden_state = dec_hidden_state[:,:,-1].unsqueeze(2) # B x H x 1

        # print("Second Hidden State Dim", dec_hidden_state.size())

        att_scores = torch.bmm(enc_outputs, dec_hidden_state) # B x S x 1
        att_scores = self.softmax(att_scores) # B x S x 1

        # print("Attention Scores Dim", att_scores.size())

        enc_outputs  = enc_outputs.permute(0, 2, 1) # B x H x S

        # print("Second Encoder Out Dim", enc_outputs.size())

        context_vects = torch.bmm(enc_outputs, att_scores).squeeze(2) # B x H x 1 -> B x H

        # print("Context Vector Dim", context_vects.size())

        dec_hidden_state = dec_hidden_state.squeeze(2) # B x H x 1 -> B x H

        # print("Third Hidden State Dim", dec_hidden_state.size())

        cur_dec = self.W1(dec_hidden_state)
        cur_enc = self.W2(context_vects)

        # print("Post Lin Layer 1 Dim", cur_dec.size())
        # print("Post Lin Layer 2 Dim", cur_enc.size())

        # print("--------------------------------")
        return self.tanh(cur_dec + cur_enc)

Starting Encoder Out Dim: torch.Size([21, 20, 256])

Starting Hidden State Dim: torch.Size([1, 21, 256])

Second Hidden State Dim: torch.Size([21, 256, 1])

Attention Scores Dim: torch.Size([21, 20, 1])

Second Encoder Out Dim: torch.Size([21, 256, 20])

Context Vector Dim: torch.Size([21, 256])

Third Hidden State Dim: torch.Size([21, 256])

Post Lin Layer 1 Dim: torch.Size([21, 256])

Post Lin Layer 2 Dim: torch.Size([21, 256])

In [26]:
class Decoder(nn.Module):
    def __init__(self, vocab_size: int, hidden_size: int, embed_size: int, layers: int, drop_rate: float, attention: Attention):
        """
        Initializes a Decoder module

        Parameters
        ----------
        vocab_size : int
            size of the decoder output
        hidden_size : int
            size of the hidden states for the LSTM
        embed_size : int
            size of the embeddings for each word
        layers : int
            number of layers of the LSTM
        drop_rate : float
            rate at which weights are probabalistically excluded from the model
        attention : Attention
            the Attention obj that the Decoder will use on its hidden states
        """
        super(Decoder, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_size)
        self.lstm_layers = nn.LSTM(input_size=embed_size, hidden_size=hidden_size, num_layers=layers, dropout=drop_rate, batch_first=True)
        self.attention = attention
        self.W0 = nn.Linear(hidden_size, vocab_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, lf_input: torch.Tensor, enc_outputs: torch.Tensor, prev_hs: torch.Tensor, prev_cs: torch.Tensor):
        """
        Feeds an input sequence, encoder outputs, a previous hidden state, and a previous cell state through the Decoder

        Parameters
        ----------
        lf_input : torch.Tensor
            the input sequence that the Decoder builds off of
        enc_outputs : torch.Tensor
            outputs of the encoder which show the end states at every time step in the LSTM
        prev_hs : torch.Tensor
            the previous Decoder hidden state (at the start this will be the Encoder's outputted hidden state)
        prev_cs : torch.Tensor
            the previous Decoder cell state (at the start this will be the Encoder's outputted cell state)

        Returns
        ----------
        tuple[torch.Tensor, torch.Tensor, torch.Tensor]
            the output of the decoder that gives probabilities for lf predictions at each time stamp, the final hidden state of the Decoder, and the final cell state of the Decoder
        """
        emb = self.embedding_layer(lf_input) # Input: list of id's of lf's | Output: Matrix of embs (len(vocab) x emb_size)
        emb = emb.unsqueeze(1)  # Ouptut: Tensor of embs that can be correctly passed into lstm (S x 1 x E)
        output, (hs, cs) = self.lstm_layers(emb, (prev_hs, prev_cs))  # output: (1 x 1 x E), hs: ()
        h_att = self.attention(enc_outputs, hs)
        prob_dists = self.softmax(self.W0(h_att))
        return prob_dists, hs, cs

In [27]:
class EncDec(nn.Module):
    def __init__(self, encoder: Encoder, decoder: Decoder, device: str):
        """
        Initializes an Encoder-Decoder module

        Parameters
        ----------
        encoder : Encoder
            Encoder module
        decoder : Decoder
            Decoder module
        device : str
            device that the model is running on
        """
        super(EncDec, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, input: torch.Tensor, tgt: torch.Tensor, teacher_force_prob=1): # src: (batch_size x list of vocab_ids), tgt: (batch_size x tgt_len x emb_size)
        """
        Feeds an input tensor and tgt tensor through the Encoder-Decoder

        Parameters
        ----------
        input : torch.Tensor
            the input sequence that gets fed through the model
        tgt : torch.Tensor
            the target sequence that gets fed through the model
        teacher_forcing_ratio : float
            the probability of a model output being used in the sequence (defaulted at 1)

        Returns
        ----------
        torch.Tensor
            the output tensor of the Encoder-Decoder model after running through both the Encoder and Decoder
        """
        enc_outputs, enc_hidden_state, enc_cell_state = self.encoder(input)
        tgt_vocab_size = self.decoder.embedding_layer.num_embeddings
        outputs = torch.zeros(tgt.shape[0], tgt.shape[1], tgt_vocab_size).to(self.device)

        lf_input = tgt[:, 0]
        hs = enc_hidden_state
        cs = enc_cell_state
        for t in range(1, tgt.shape[1]):
            probs, hs, cs = self.decoder(lf_input, enc_outputs, hs, cs)
            outputs[:, t, :] = probs
            if torch.rand(1) < teacher_force_prob:
                lf_input = tgt[:, t]
            else:
                lf_input = torch.argmax(probs, dim=1)
        return outputs

In [28]:
def create_model():
    """
    Returns
    ----------
    Model
        Encoder-Decoder model!
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    hidden_dim = 256
    emb_dim = 128
    layers = 1
    drop_rate = 0.02

    encoder = Encoder(QUERY_VOCAB_LEN, hidden_dim, emb_dim, layers, drop_rate)
    attention = Attention(hidden_dim)
    decoder = Decoder(LF_VOCAB_LEN, hidden_dim, emb_dim, layers, drop_rate, attention)
    model = EncDec(encoder, decoder, device).to(device)
    return model

# TODO: Training and testing loops

In [29]:
LF_SOS_INDEX = lf_word2idx['<s>']
LF_EOS_INDEX = lf_word2idx['</s>']
LF_PAD_INDEX = lf_word2idx['<PAD>']

In [30]:
def train_epoch(model: nn.Module,
                train_dataloader: DataLoader,
                loss_fn: nn.Module,
                optimizer: torch.optim.Optimizer,
                device: str) -> float:
    """
    Trains the inputted model using the provided data, optimizer, loss
    function, and device for one epoch. Returns the average loss of the epoch.

    Parameters
    ----------
    model : nn.Module
        model to train
    train_dataloader : DataLoader
        training data
    loss_fn : nn.Module
        loss function to use with training
    optimizer : torch.optim.Optimizer
        optimizer to use with training
    device : str
        device that the model is running on

    Returns
    ----------
    float
        epoch average loss
    """
    model.train()
    total_loss = 0

    for input, tgt in train_dataloader:
        input, tgt = input.to(device).permute(1, 0), tgt.to(device).permute(1, 0)
        outputs = model(input, tgt)
        optimizer.zero_grad()

        outputs = outputs[:, 1:, :]
        tgt = tgt[:, 1:]
        batch_size, seq_len, vocab_size = outputs.size()
        flattened_outputs = outputs.reshape(batch_size * seq_len, vocab_size)
        flattened_tgt = tgt.reshape(batch_size * seq_len)

        loss = loss_fn(flattened_outputs, flattened_tgt)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    return total_loss / len(train_dataloader)

def train(model: nn.Module,
          train_dataloader : DataLoader,
          loss_fn: nn.Module,
          optimizer: torch.optim.Optimizer,
          n_epochs: int, device: str):
    """
    Trains the inputted model using the provided data, optimizer, loss
    function, and device for n epochs. Prints the average loss of each epoch.

    Parameters
    ----------
    model : nn.Module
        model to train
    train_dataloader : DataLoader
        training data
    loss_fn : nn.Module
        loss function to use with training
    optimizer : torch.optim.Optimizer
        optimizer to use with training
    n_epochs : int
        number of epochs to train for
    device : str
        device that the model is running on

    Returns
    ----------
    nn.Module
        trained model
    """

    model = model.to(device)
    for epoch in range(1, n_epochs+1):
        avg_loss = train_epoch(model, train_dataloader, loss_fn, optimizer, device)
        print("Epoch:", epoch, "Loss:", avg_loss)
    return model

In [31]:
def evaluate(model: nn.Module, dataloader: DataLoader, device: str="cuda") -> tuple[int, int]:
    """
    Evaluates your model!

    Parameters
    ----------
    model : nn.Module
        your model!
    dataloader : DataLoader
        a dataloader of the testing data from build_dataloaders
    device : str
        device that the model is running on

    Returns
    ----------
    tuple[int, int]
        per-token accuracy and exact_match accuracy
    """
    model.to(device)
    model.eval()
    tot_tokens = 0
    tot_correct = 0
    tot_exact_matches = 0
    tot_sequences = 0

    with torch.no_grad():
        for input, tgt in dataloader:
            input, tgt = input.to(device).permute(1, 0), tgt.to(device).permute(1, 0)
            outputs = model(input, tgt)
            greedy_predict = outputs.argmax(2)

            end_idxs = []
            for i in range(tgt.size(0)):
                seq = tgt[i, :]
                end_idx = torch.where(seq == LF_EOS_INDEX)[0]
                end_idxs.append(end_idx.item())
                break
            end_idxs = torch.tensor(end_idxs)

            for batch_idx in range(tgt.size(0)):
                model_comp = greedy_predict[batch_idx, :end_idxs[batch_idx] + 1]
                tgt_comp = tgt[batch_idx, :end_idxs[batch_idx] + 1]
                fully_correct = True
                for i in range(tgt_comp.size(0)):
                    if(model_comp[i] == tgt_comp[i]):
                        tot_correct += 1
                    else:
                        fully_correct = False
                    tot_tokens += 1
                if(fully_correct):
                    tot_exact_matches += 1
                tot_sequences += 1
    per_token_accuracy = tot_correct / tot_tokens
    exact_match_accurancy = tot_exact_matches / tot_sequences
    return per_token_accuracy, exact_match_accurancy

# Run this!

In [50]:
def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    jobs_train, jobs_test = build_datasets()
    dataloader_train, dataloader_test = build_dataloaders(jobs_train, jobs_test, train_batch_size=21)
    model = create_model()
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.003)
    loss_fn = nn.NLLLoss()
    model = train(model, dataloader_train, loss_fn, optimizer, n_epochs=5, device=device)
    test_per_token_accuracy, test_exact_match_accuracy = evaluate(model, dataloader_test, device=device)
    print(f'Test Per-token Accuracy: {test_per_token_accuracy}')
    print(f'Test Exact-match Accuracy: {test_exact_match_accuracy}')
main()

Epoch: 1 Loss: 0.8169908492515484
Epoch: 2 Loss: 0.24065837015708288
Epoch: 3 Loss: 0.18033159400026003
Epoch: 4 Loss: 0.12182168817768495
Epoch: 5 Loss: 0.08884651617457469
Test Per-token Accuracy: 0.9392265193370166
Test Exact-match Accuracy: 0.5357142857142857
